### If it was helpful please upvote 

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


import os
from fastai.vision import *
from fastai import *
import matplotlib.pyplot as plt
import seaborn as sns
from functools import partial
from tqdm.notebook import tqdm
import gc
from pylab import imread,subplot,imshow,show
%matplotlib inline

In [ ]:
path = "/kaggle/input/breast-cancer-image-data/project/"

In [ ]:
size = 224
bs = 64
data = ImageDataBunch.from_folder(path, 
                                  ds_tfms=get_transforms(max_rotate=0.1,max_lighting=0.15),
                                  valid_pct=0.2, 
                                  size=size, 
                                  bs=bs)


In [ ]:
data.show_batch(rows=4)

### Total classes, length of train, validation and test set

In [ ]:
len(data.classes)

# Using a pretrained ResNet50 model

## with metrics = f1_score 
>average = macro 


* because there is class imbalance in the data set

In [ ]:
fb = FBeta()
fb.average='macro'

In [ ]:
arch = models.resnet50

In [ ]:
#!mkdir -p /tmp/.cache/torch/checkpoints/
#!cp /kaggle/input/resnet50/resnet50.pth  /root/.cache/torch/checkpoints/resnet50-19c8e357.pth

# Using mixed precision training

* Mixed precision training utilizes half-precision to speed up training, achieving the same accuracy in some cases as single-precision training using the same hyper-parameters. 
* Memory requirements are also reduced, allowing larger models and minibatches

In [ ]:
try:
    learn = cnn_learner(data, arch, metrics = [fb],model_dir='/kaggle/working').to_fp16()
except:
    !mkdir -p /tmp/.cache/torch/checkpoints/
    !cp /kaggle/input/resnet50/resnet50.pth  /root/.cache/torch/checkpoints/resnet50-19c8e357.pth
    
    learn = cnn_learner(data, arch, metrics = [fb],model_dir='/kaggle/working').to_fp16()

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
gc.collect()

# Model Summary

In [ ]:
learn.summary()

In [ ]:
lr = 1e-2

In [ ]:
gc.collect()

In [ ]:
learn.fit_one_cycle(6,lr,moms=(0.9,0.8))

# Classification Interpretation

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

In [ ]:
interp.plot_top_losses(12,figsize=(20,8))

# Most Confused

In [ ]:
interp.most_confused(min_val=3)

In [ ]:
learn.save('model1')


In [ ]:
learn.export('/kaggle/working/breast.pkl')

In [ ]:
img = open_image('/kaggle/input/breast-cancer-image-data/project/benin/mdb002.jpg')
print(learn.predict(img)[0])
img